In [1]:
import pandas as pd
from collections import defaultdict
from intervaltree import IntervalTree
import ast

In [2]:
# ==========================================================
# STEP 1: Load Input Data
# ==========================================================

print("Loading CW data...")
cw_df = pd.read_excel('cw_cleaned.xlsx')

print("Loading Perform data...")
perform_df = pd.read_excel('perform_datetime.xlsx')

print("Loading Event data...")
event_df = pd.read_excel('event_interpolated.xlsx')

# Handle datetime columns 
cw_df['start'] = pd.to_datetime(cw_df['start'], errors='coerce').dt.tz_localize(None)
cw_df['end'] = pd.to_datetime(cw_df['end'], errors='coerce').dt.tz_localize(None)
perform_df['result_from'] = pd.to_datetime(perform_df['result_from'], errors='coerce').dt.tz_localize(None)
perform_df['result_to'] = pd.to_datetime(perform_df['result_to'], errors='coerce').dt.tz_localize(None)
event_df['occurred_at'] = pd.to_datetime(event_df['occurred_at'], format="ISO8601").dt.tz_localize(None)

Loading CW data...
Loading Perform data...
Loading Event data...


In [3]:
# ==========================================================
# STEP 2: Align and Filter Datasets Using Asset IDs
# ==========================================================
print("Filtering Necessary Rows from Perform and Event Data...")

# # Define the vehicle types to filter for
# vehicle_types = ['FL', 'ARC']  

# # Filter cw_df to include only rows with the specified vehicle types
# cw_df = cw_df[cw_df['vehicleType'].isin(vehicle_types)].copy()

# Convert string representation of list to actual list
perform_df['asset_ids'] = perform_df['asset_ids'].apply(ast.literal_eval)
# Extract the single string from the list (since there is only 1 item per row)
perform_df['asset_ids'] = perform_df['asset_ids'].apply(lambda x: x[0] if isinstance(x, list) else x)

# Create a dictionary mapping asset_name -> asset_ids
mapping_dict = dict(zip(perform_df['asset_name'], perform_df['asset_ids']))

# Map cw_df['truck'] using this dictionary to get perf_asset_ids
cw_df['perf_asset_ids'] = cw_df['truck'].map(mapping_dict)

# Extract unique perf_asset_ids values from cw_df (excluding NaNs)
valid_asset_ids = cw_df['perf_asset_ids'].dropna().unique()

# Filter perform_df where asset_ids are in valid_asset_ids
perform_df_filtered = perform_df[perform_df['asset_ids'].isin(valid_asset_ids)].copy()

# Filter event_df to include only events matching the asset IDs of interest
event_df = event_df[event_df['asset_id'].isin(valid_asset_ids)].copy()


Filtering Necessary Rows from Perform and Event Data...


In [4]:
# ==========================================================
# STEP 3: Group Perform columns For Overlap Calculation
# ==========================================================
print("\nGrouping Perform columns for overlap calculation...")

# Categorize into WEIGHTED_AVG_COLS, CATEGORICAL_COLS, and LINEAR_SCALE_COLS
WEIGHTED_AVG_COLS = [
    'total_rating', 'coasting_rating', 'acceleration_pedal_rating', 'braking_pedal_rating',
    'cruise_control_rating', 'overspeed_rating', 'harsh_acceleration_rating', 'harsh_braking_rating',
    'average_speed', 'average_rpm', 'fuel_efficiency', 'operating_conditions_score',
    'overspeed_percentage', 'kickdown_percentage', 'excessive_idling_rating',
    'ambient_temperature_min', 'ambient_temperature_max'
]

excluded_cols = set(WEIGHTED_AVG_COLS + ['result_from', 'result_to', 'asset_name', 'asset_ids'])

LINEAR_SCALE_COLS = []
CATEGORICAL_COLS = []

for col in perform_df.columns:
    if col in excluded_cols:
        continue  # skip weighted avg and excluded cols
    
    # Skip columns that are empty or all NaN
    if perform_df[col].dropna().empty:
        continue
    
    if pd.api.types.is_numeric_dtype(perform_df[col]):
        LINEAR_SCALE_COLS.append(col)
    else:
        CATEGORICAL_COLS.append(col)

# print("\nWEIGHTED_AVG_COLS:", WEIGHTED_AVG_COLS)
# print("\nLINEAR_SCALE_COLS:", LINEAR_SCALE_COLS)
# print("\nCATEGORICAL_COLS:", CATEGORICAL_COLS)


Grouping Perform columns for overlap calculation...


In [5]:
# ==========================================================
# STEP 4: Merge Perform data with CW data Using IntervalTree
# ==========================================================
print("Building IntervalTree from Performance data...")

perf_intervals = []
for idx, row in perform_df.iterrows():
    perf_intervals.append((
        row['result_from'].timestamp(),
        row['result_to'].timestamp(),
        idx
    ))

perf_tree = IntervalTree()
for start_ts, end_ts, idx in perf_intervals:
    perf_tree[start_ts:end_ts] = idx

print("Merging Performance data with CW data...")

# Define Overlap Time Calculation Function
def calculate_overlap(start1, end1, start2, end2):
    overlap_start = max(start1, start2)
    overlap_end = min(end1, end2)
    delta = overlap_end - overlap_start
    return delta if delta > pd.Timedelta(0) else pd.Timedelta(0)

# Define merge Function Using IntervalTree
def merge_tour_performance_data_intervaltree(cw_df, perf_df, perf_tree):
    merged_rows = []

    for _, segment in cw_df.iterrows():
        merged_row = segment.to_dict()

        linear_sums = {col: 0.0 for col in LINEAR_SCALE_COLS}
        weighted = {col: {'sum': 0.0, 'duration': 0.0} for col in WEIGHTED_AVG_COLS}
        categorical_values = defaultdict(set)

        seg_start_ts = segment['start'].timestamp()
        seg_end_ts = segment['end'].timestamp()

        overlapping_perf_intervals = perf_tree.overlap(seg_start_ts, seg_end_ts)

        for interval in overlapping_perf_intervals:
            perf_idx = interval.data
            perf = perf_df.loc[perf_idx]

            if perf['asset_name'] != segment['truck']:
                continue

            overlap_dur = calculate_overlap(segment['start'], segment['end'], perf['result_from'], perf['result_to'])
            if overlap_dur.total_seconds() <= 0:
                continue

            perf_dur = perf['result_to'] - perf['result_from']
            overlap_ratio = overlap_dur.total_seconds() / perf_dur.total_seconds()

            for col in LINEAR_SCALE_COLS:
                if pd.notna(perf[col]):
                    linear_sums[col] += perf[col] * overlap_ratio

            for col in WEIGHTED_AVG_COLS:
                if pd.notna(perf[col]):
                    weighted[col]['sum'] += perf[col] * overlap_dur.total_seconds()
                    weighted[col]['duration'] += overlap_dur.total_seconds()

            for col in CATEGORICAL_COLS:
                if pd.notna(perf[col]):
                    categorical_values[col].add(str(perf[col]))

        for col in LINEAR_SCALE_COLS:
            merged_row[f'perf_{col}'] = linear_sums[col] if linear_sums[col] != 0 else None

        for col in WEIGHTED_AVG_COLS:
            dur = weighted[col]['duration']
            merged_row[f'perf_{col}'] = weighted[col]['sum'] / dur if dur > 0 else None

        for col in CATEGORICAL_COLS:
            values = sorted(categorical_values[col])
            merged_row[f'perf_{col}'] = ', '.join(values) if values else None

        merged_rows.append(merged_row)

    return pd.DataFrame(merged_rows)

# Apply merge function
cw_perform_df = merge_tour_performance_data_intervaltree(cw_df, perform_df, perf_tree)


Building IntervalTree from Performance data...
Merging Performance data with CW data...


In [6]:
# ==========================================================
# STEP 5: Merge with Event Data 
# ==========================================================

print("Merging Event data with CW-Perform data...")

event_df = event_df.add_prefix('event_')

# Sort both dataframes by datetime to use merge_asof
cw_perform_df = cw_perform_df.sort_values('start').reset_index(drop=True)
event_df = event_df.sort_values('event_occurred_at').reset_index(drop=True)


merged_parts = []

for idx, row in cw_perform_df.iterrows():
    mask = (
        (event_df['event_occurred_at'] >= row['start']) &
        (event_df['event_occurred_at'] <= row['end']) &
        (event_df['event_asset_id'] == row['perf_asset_ids'])
    )
    matching_events = event_df.loc[mask].copy()
    if not matching_events.empty:
        repeated_cw = pd.concat([row.to_frame().T]*len(matching_events), ignore_index=True)
        merged = pd.concat([repeated_cw.reset_index(drop=True), matching_events.reset_index(drop=True)], axis=1)
        merged_parts.append(merged)

cw_perform_event_df = pd.concat(merged_parts, ignore_index=True)


Merging Event data with CW-Perform data...


In [7]:
# ==========================================================
# STEP 7: Save Final Output
# ==========================================================
print("Saving final output...")

# Save the cleaned dataframe
output_file = 'cw_perform_event.xlsx'
cw_perform_event_df.to_excel(output_file, index=False)

Saving final output...


/Users/tunkarlu/miniconda3/envs/myenv/lib/python3.12/site-packages/xlsxwriter/worksheet.py:1250: UserWarning: Ignoring URL 'https://feed.iot-events.rio.cloud/iot-events?after=01JKAWE9YC67NM1ZMQ1J3T0021&limit=1000' since it exceeds Excel's limit of 65,530 URLs per worksheet.
  warn(
/Users/tunkarlu/miniconda3/envs/myenv/lib/python3.12/site-packages/xlsxwriter/worksheet.py:1250: UserWarning: Ignoring URL 'https://feed.iot-events.rio.cloud/iot-events?after=01JKAVQXKB40EK1YFR73A50JEE&limit=1000' since it exceeds Excel's limit of 65,530 URLs per worksheet.
  warn(
/Users/tunkarlu/miniconda3/envs/myenv/lib/python3.12/site-packages/xlsxwriter/worksheet.py:1250: UserWarning: Ignoring URL 'https://feed.iot-events.rio.cloud/iot-events?after=01JKAW3BAP7K5M1VFHZ2VE1DJ5&limit=1000' since it exceeds Excel's limit of 65,530 URLs per worksheet.
  warn(
/Users/tunkarlu/miniconda3/envs/myenv/lib/python3.12/site-packages/xlsxwriter/worksheet.py:1250: UserWarning: Ignoring URL 'https://feed.iot-events.rio